# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/9851/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

['/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-10.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-11.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-12.csv',
 '/home/runner/work/dask-exa

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,956,Alice,-0.4807276944690151,0.23356788043142274
2000-01-01 00:00:01,1021,Sarah,-0.3484081198819542,-0.2911518751131208
2000-01-01 00:00:02,1025,Victor,0.8476634344555833,-0.16005592140365543
2000-01-01 00:00:03,993,Ray,0.08069276228720024,0.20530953887879644
2000-01-01 00:00:04,1042,Frank,0.8084479486913967,-0.06031440805396504
2000-01-01 00:00:05,1019,Michael,-0.44176535520155413,-0.3175961488952834
2000-01-01 00:00:06,1000,Tim,0.265404109447662,-0.4416574643407474
2000-01-01 00:00:07,987,Bob,0.8133956909053868,-0.5520896178063626
2000-01-01 00:00:08,1010,Ingrid,0.48954392824103965,-0.5004332236304483


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1010,Norbert,0.6388864196859816,0.4404903971820411
2000-01-30 00:00:01,947,Sarah,-0.9389995612083972,-0.3973788781388472
2000-01-30 00:00:02,1005,Quinn,-0.48404866201305374,-0.04634673894903685
2000-01-30 00:00:03,1035,Jerry,0.9583721569302004,-0.20237635709714885
2000-01-30 00:00:04,1012,Hannah,0.2657948312028029,-0.6734865104543839
2000-01-30 00:00:05,928,Quinn,0.07751635230308618,-0.14444273590061463
2000-01-30 00:00:06,1023,Quinn,-0.5327028173448654,-0.7794600508434462
2000-01-30 00:00:07,1010,Jerry,0.28484348676422333,-0.13229459801520815
2000-01-30 00:00:08,995,George,0.9336344851309977,-0.9433452431671332


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,956,Alice,-0.480728,0.233568
1,2000-01-01 00:00:01,1021,Sarah,-0.348408,-0.291152
2,2000-01-01 00:00:02,1025,Victor,0.847663,-0.160056
3,2000-01-01 00:00:03,993,Ray,0.080693,0.205310
4,2000-01-01 00:00:04,1042,Frank,0.808448,-0.060314


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,956,Alice,-0.480728,0.233568
1,2000-01-01 00:00:01,1021,Sarah,-0.348408,-0.291152
2,2000-01-01 00:00:02,1025,Victor,0.847663,-0.160056
3,2000-01-01 00:00:03,993,Ray,0.080693,0.205310
4,2000-01-01 00:00:04,1042,Frank,0.808448,-0.060314


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.96 s, sys: 587 ms, total: 4.55 s
Wall time: 3.14 s


name
Alice       0.000391
Bob        -0.000239
Charlie     0.004081
Dan        -0.000132
Edith      -0.000661
Frank       0.003799
George      0.002192
Hannah      0.003944
Ingrid     -0.001456
Jerry       0.002225
Kevin      -0.001252
Laura       0.000384
Michael    -0.000504
Norbert    -0.000153
Oliver      0.001463
Patricia    0.002335
Quinn      -0.001586
Ray         0.001449
Sarah      -0.003739
Tim         0.002277
Ursula     -0.000669
Victor     -0.000065
Wendy       0.001108
Xavier     -0.001508
Yvonne      0.000187
Zelda      -0.000180
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.26 s, sys: 115 ms, total: 1.37 s
Wall time: 1.03 s


name
Alice       0.000391
Bob        -0.000239
Charlie     0.004081
Dan        -0.000132
Edith      -0.000661
Frank       0.003799
George      0.002192
Hannah      0.003944
Ingrid     -0.001456
Jerry       0.002225
Kevin      -0.001252
Laura       0.000384
Michael    -0.000504
Norbert    -0.000153
Oliver      0.001463
Patricia    0.002335
Quinn      -0.001586
Ray         0.001449
Sarah      -0.003739
Tim         0.002277
Ursula     -0.000669
Victor     -0.000065
Wendy       0.001108
Xavier     -0.001508
Yvonne      0.000187
Zelda      -0.000180
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.2 s, sys: 80.1 ms, total: 1.28 s
Wall time: 997 ms


name
Alice       0.000391
Bob        -0.000239
Charlie     0.004081
Dan        -0.000132
Edith      -0.000661
Frank       0.003799
George      0.002192
Hannah      0.003944
Ingrid     -0.001456
Jerry       0.002225
Kevin      -0.001252
Laura       0.000384
Michael    -0.000504
Norbert    -0.000153
Oliver      0.001463
Patricia    0.002335
Quinn      -0.001586
Ray         0.001449
Sarah      -0.003739
Tim         0.002277
Ursula     -0.000669
Victor     -0.000065
Wendy       0.001108
Xavier     -0.001508
Yvonne      0.000187
Zelda      -0.000180
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html